In [7]:
import cv2
import numpy as np

gray_img = True
# Read the image
file = "side-walk-2"
img = cv2.imread(f"{file}.jpg")
# img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) if gray else img

# Resize the image
scale = 0.3
small_img = cv2.resize(img, None, fx=scale, fy=scale)

# Convert to grayscale
if gray_img:
    gray = cv2.cvtColor(small_img, cv2.COLOR_BGR2GRAY)
else:
    gray = small_img

# Create a window and trackbars
cv2.namedWindow('window')
cv2.createTrackbar('lower', 'window', 0, 255, lambda x: None)
cv2.createTrackbar('upper', 'window', 0, 255, lambda x: None)

while True:
    # Get the current positions of the trackbars
    low = cv2.getTrackbarPos('lower', 'window')
    high = cv2.getTrackbarPos('upper', 'window')

    # Apply Canny edge detection
    size = (5, 5)
    blurred = cv2.GaussianBlur(gray, size, 0)
    edge = cv2.Canny(blurred, low, high)

    # Detect lines using Hough transform
    lines = cv2.HoughLinesP(edge, 1, np.pi / 180, threshold=100, minLineLength=20, maxLineGap=20)

    # Draw lines on the original image
    line_img = small_img.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Show the original image with detected lines
    cv2.imshow('window', line_img)

    # Check for 'q' key press to exit the loop and save the result image and parameter values
    key = cv2.waitKey(1)
    if key == ord('q'):
        # Save the result image
        cv2.imwrite(f'res/{file}-{gray_img}-{size[0]}{size[1]}-{low}-{high}.jpg', line_img)
        
        # Save the parameter values to a text file
        with open('canny_parameters-2.txt', 'w') as f:
            f.write(f'Lower Threshold: {low}\n')
            f.write(f'Higher Threshold: {high}\n')
        
        cv2.destroyAllWindows()
        break